# NewsAPI Testing

In [1]:
from newsapi import NewsApiClient
import newsapi as news
from dotenv import load_dotenv
import os

load_dotenv(override=True)

True

In [2]:
news_api_key = os.getenv("NEWS_API_KEY")
if not news_api_key:
    print("NEWS_API_KEY environment variable not set.")

# get_top_headlines

In [3]:
print(news.const.sort_method)
print(news.const.categories)

{'relevancy', 'popularity', 'publishedAt'}
{'entertainment', 'technology', 'science', 'sports', 'health', 'general', 'business'}


In [16]:
newsapi = NewsApiClient(api_key=news_api_key)

top_headlines = newsapi.get_top_headlines(country='us', language='en', category="business", q="trump")

top_headlines

{'status': 'ok',
 'totalResults': 7,
 'articles': [{'source': {'id': 'the-washington-post',
    'name': 'The Washington Post'},
   'author': 'Lauren Kaori Gurley',
   'title': 'Trump’s pick for BLS commissioner endorses suspending monthly jobs reports - The Washington Post',
   'description': '“The BLS should suspend issuing the monthly job reports,” E.J. Antoni, President Donald Trump’s pick to lead the federal statistical agency, said on Fox News Digital.',
   'url': 'https://www.washingtonpost.com/business/2025/08/12/bls-antoni-suspend-jobs-report/',
   'urlToImage': 'https://www.washingtonpost.com/wp-apps/imrs.php?src=https://arc-anglerfish-washpost-prod-washpost.s3.amazonaws.com/public/CZQWAUPQUZAT3KBKAM7S2OHSBM_size-normalized.jpg&w=1440',
   'publishedAt': '2025-08-12T21:46:09Z',
   'content': 'The BLS should suspend issuing the monthly job reports, E.J. Antoni, President Trumps pick to lead the federal statistical agency, said in an interview with Fox News Digital.\r\nUpdated A

In [5]:
articles = top_headlines["articles"]

articles

[{'source': {'id': None, 'name': 'Boredpanda.com'},
  'author': 'Analice Couto',
  'title': "Mom Ends Up in Urgent Care After Popping Pimple In The “Triangle Of Death”—Here's Why Doctors Warn Against It - Bored Panda",
  'description': '"After seeing this, I feel like I\'ve cheated d*ath 100x."',
  'url': 'https://www.boredpanda.com/i-popped-a-pimple-in-the-triangle-of-death-and-ended-up-in-urgent-care/',
  'urlToImage': 'https://www.boredpanda.com/blog/wp-content/uploads/2025/08/fb_image_689ba4b07e300.png',
  'publishedAt': '2025-08-12T21:06:10Z',
  'content': 'When Lish Marie noticed a cyst-like pimple just under her nostril, she didn’t think twice before giving it a squeeze.\r\nWithin hours, the mom of three was sitting in urgent care, holding four differen… [+5600 chars]'},
 {'source': {'id': 'cnn', 'name': 'CNN'},
  'author': None,
  'title': 'Plague mysteriously spread from Europe to Asia thousands of years ago. Scientists now think they know how it was transmitted - CNN',
  'des

In [6]:
source = articles[1]["source"]["id"]

print(source)

cnn


In [7]:
cnn_and_fox_articles = [article for article in articles if article["source"]["id"] == "fox-news" or article["source"]["id"] == "cnn"]

cnn_and_fox_articles

[{'source': {'id': 'cnn', 'name': 'CNN'},
  'author': None,
  'title': 'Plague mysteriously spread from Europe to Asia thousands of years ago. Scientists now think they know how it was transmitted - CNN',
  'description': 'Researchers recovered the first Yersinia pestis bacteria genome from a Bronze Age animal. It reveals how a plague spread in Eurasia for thousands of years, experts say.',
  'url': 'https://www.cnn.com/2025/08/12/science/plague-livestock-transmission-black-death',
  'urlToImage': 'https://media.cnn.com/api/v1/images/stellar/prod/excavation.jpg?c=16x9&q=w_800,c_fill',
  'publishedAt': '2025-08-12T20:23:00Z',
  'content': 'For thousands of years, a disease repeatedly struck ancient Eurasia, quickly spreading far and wide. The bite of infected fleas that lived on rats passed on the plague in its most infamous form the B… [+6229 chars]'},
 {'source': {'id': 'fox-news', 'name': 'Fox News'},
  'author': 'Melissa Rudy',
  'title': 'Study reveals why ‘super agers’ maintain ‘o

In [9]:
from agents import Agent, Runner, trace, function_tool
import asyncio

In [24]:
from typing import List, Optional

@function_tool
def get_headlines(
    sources: Optional[List[str]] = None,
    category: Optional[str] = None,
    query: Optional[str] = None
) -> List[dict]:
    """
    Returns news headlines from the provided sources, category, and/or query.

    Args:
        sources (Optional[List[str]]): A list of news source IDs to filter by.
        category (Optional[str]): The category of news to filter by (e.g., 'business', 'technology').
        query (Optional[str]): A keyword or phrase to search for in headlines.

    Returns:
        List[dict]: A list of articles, each represented as a dictionary.
    """
    newsapi = NewsApiClient(api_key=news_api_key)
    top_headlines = newsapi.get_top_headlines(country='us', language='en', category=category, q=query)
    articles = top_headlines["articles"]
    if sources:
        return [article for article in articles if article["source"]["id"] in sources]
    return articles
    

In [25]:
instructions = """
You are an excellent writer of an email news digest. You carefully listen to the desires of your customer and write them a personalized news digest including only the types of news that they are interested in.
You always make sure to cite your sources by including the link to the original article and write without bias and in a neat markdown format. 
You have a history of using reliable sources to include only the most up-to-date news.
Always use the get_headlines tool to get up-to-date news headlines.
"""

news_agent = Agent(
    name="News Agent",
    tools=[get_headlines],
    instructions=instructions,
    model="gpt-4.1"
)

In [27]:
from IPython.display import display, Markdown

prompt = """
My name is Arnav and I'm looking to hear about what's new in the health space, specifically news about diseases.
"""

with trace("News Digest"):
    result = await Runner.run(news_agent, input=prompt)
    display(Markdown(result.final_output))

Hi Arnav,

Here are the latest updates in the health space, focusing on diseases:

---

### 1. Legionnaires’ Disease Cluster in New York City
A cluster of Legionnaires’ disease in Harlem, New York City, has resulted in 3 deaths and at least 90 illnesses. Health officials are investigating the source and reminding the public about symptoms and prevention.  
Read more: [Yahoo News](https://www.yahoo.com/news/us/article/legionnaires-disease-cluster-in-new-york-city-causes-3-deaths-sickens-90-people-what-is-it-and-how-do-you-get-it-195941423.html)

---

### 2. The Hidden Epidemic of Lung Cancer in Sub-Saharan Africa
Lung cancer, the world’s deadliest cancer, is trending as a “hidden epidemic” in parts of sub-Saharan Africa. Low reported rates actually mask underdiagnosis and limited data, according to experts, and many cases go untreated.  
Full story: [NPR](https://www.npr.org/sections/goats-and-soda/2025/08/12/g-s1-77203/lung-cancer-africa)

---

### 3. Unique Relationship Between Centenarians and Disease
A new study explores how people who live to 100 years or beyond have distinct patterns of disease—often delaying the onset or experiencing fewer chronic illnesses compared to others. The biological and lifestyle factors behind this phenomenon are being closely examined.  
Details here: [ScienceAlert](https://www.sciencealert.com/people-who-live-to-100-have-a-unique-relationship-with-disease)

---

### 4. IVF Breakthrough: Babies Born Free of Genetic Disease Using DNA from Three People
A pioneering IVF procedure in the UK has allowed eight healthy babies to be born free from inherited mitochondrial diseases by using DNA from three individuals. This breakthrough offers hope to families at risk of passing on genetic disorders.  
Learn more: [SciTechDaily](https://scitechdaily.com/dna-from-three-people-creates-healthy-babies-free-of-genetic-disease/)

---

Let me know if you’d like to dive deeper into any of these stories or pivot to other topics within health!

Best,  
Your Personalized News Digest

# get_sources

In [28]:
sources = newsapi.get_sources()

sources

{'status': 'ok',
 'sources': [{'id': 'abc-news',
   'name': 'ABC News',
   'description': 'Your trusted source for breaking news, analysis, exclusive interviews, headlines, and videos at ABCNews.com.',
   'url': 'https://abcnews.go.com',
   'category': 'general',
   'language': 'en',
   'country': 'us'},
  {'id': 'abc-news-au',
   'name': 'ABC News (AU)',
   'description': "Australia's most trusted source of local, national and world news. Comprehensive, independent, in-depth analysis, the latest business, sport, weather and more.",
   'url': 'https://www.abc.net.au/news',
   'category': 'general',
   'language': 'en',
   'country': 'au'},
  {'id': 'aftenposten',
   'name': 'Aftenposten',
   'description': 'Norges ledende nettavis med alltid oppdaterte nyheter innenfor innenriks, utenriks, sport og kultur.',
   'url': 'https://www.aftenposten.no',
   'category': 'general',
   'language': 'no',
   'country': 'no'},
  {'id': 'al-jazeera-english',
   'name': 'Al Jazeera English',
   'desc

In [30]:
sources_list = sources["sources"]

american_sources = [source for source in sources_list if source["country"] == "us" and source["language"] == "en"]

american_sources

[{'id': 'abc-news',
  'name': 'ABC News',
  'description': 'Your trusted source for breaking news, analysis, exclusive interviews, headlines, and videos at ABCNews.com.',
  'url': 'https://abcnews.go.com',
  'category': 'general',
  'language': 'en',
  'country': 'us'},
 {'id': 'al-jazeera-english',
  'name': 'Al Jazeera English',
  'description': 'News, analysis from the Middle East and worldwide, multimedia and interactives, opinions, documentaries, podcasts, long reads and broadcast schedule.',
  'url': 'https://www.aljazeera.com',
  'category': 'general',
  'language': 'en',
  'country': 'us'},
 {'id': 'ars-technica',
  'name': 'Ars Technica',
  'description': "The PC enthusiast's resource. Power users and the tools they love, without computing religion.",
  'url': 'https://arstechnica.com',
  'category': 'technology',
  'language': 'en',
  'country': 'us'},
 {'id': 'associated-press',
  'name': 'Associated Press',
  'description': 'The AP delivers in-depth coverage on the internat

In [31]:
@function_tool
def get_sources() -> List[dict]:
    """
    Returns a list of US-based, English-language news sources.

    Returns:
        List[dict]: A list of source objects from NewsAPI that are in the US and in English.
    """
    newsapi = NewsApiClient(api_key=news_api_key)
    sources = newsapi.get_sources()
    sources_list = sources.get("sources", [])
    american_sources = [
        source for source in sources_list
        if source.get("country") == "us" and source.get("language") == "en"
    ]
    return american_sources
    

In [32]:
instructions = """
You are an excellent writer of an email news digest. You carefully listen to the desires of your customer and write them a personalized news digest including only the types of news that they are interested in.
You always make sure to cite your sources by including the link to the original article and write without bias and in a neat markdown format. 
You have a history of choosing the best sources for the user based on the type of news they like, even if they don't mention any source preferences.

Always use the get_sources tool to get a list of possible news sources you can get news from and information about them.
Always use the get_headlines tool to get up-to-date news headlines.
"""

news_agent = Agent(
    name="News Agent",
    tools=[get_headlines, get_sources],
    instructions=instructions,
    model="gpt-4.1"
)

In [36]:
from IPython.display import display, Markdown

prompt = """
My name is Arnav and I'm looking to hear about what's new in the technology space.
"""

with trace("News Digest"):
    result = await Runner.run(news_agent, input=prompt)
    display(Markdown(result.final_output))

Hi Arnav,

Here's your quick digest of the latest in the technology space, featuring stories from some of the best tech news sources:

---

### 1. Google Vet Raises $8M for AI Group Chat Startup  
David Petrou, a founding engineer behind Google Goggles and Google Glass, has raised $8 million for his new startup, Continua. The company aims to enrich group chats by introducing AI agents, potentially transforming the way teams and communities collaborate online.  
[Read more on TechCrunch](https://techcrunch.com/2025/08/12/google-vet-raises-8m-for-continua-to-bring-ai-agents-to-group-chats/)

---

### 2. Apple Cinemas Responds to Apple Lawsuit  
Apple Cinemas, the theater chain, is pushing back against a lawsuit from Apple Inc. over their brand name. The company said, "We are committed to defending our brand, our history, and our continued right to operate as Apple Cinemas."  
[Details at The Verge](https://www.theverge.com/news/758192/apple-cinemas-lawsuit-response)

---

### 3. Blackwater Founder’s Privacy-First Phone Made in USA  
Erik Prince, founder of Blackwater, is supporting a new "privacy-first" device called the Unplugged UP Phone. The phone emphasizes user privacy and, starting later this year, will be manufactured in the USA.  
[Explore the story on The Verge](https://www.theverge.com/mobile/758073/unplugged-up-phone-erik-prince-blackwater)

---

### 4. Google Lets Users Choose Top News Sources  
Google is introducing a feature called “Preferred Sources” in the U.S. and India. This allows users to choose their preferred news sites, displaying more stories from those sources in search results.  
[Read about it on TechCrunch](https://techcrunch.com/2025/08/12/google-will-now-allow-you-define-your-top-sources-for-search-results/)

---

Let me know if you’d like news from a particular tech domain (AI, gadgets, startups, security, etc.) or prefer deeper coverage from a specific source!

Best,  
Your News Digest Assistant

# get_everything